# About this kernel
This kernel may (or may not) be helpful in your long and often tedious machine learning journey. This kernel is easily understandable to the beginner like me.  This verbosity tries to explain everything I could possibly know. Once you get through the notebook, you can find this useful and straightforward. I attempted to explain things as simple as possible.

In this kernel, I'm going to attempt the only Machine learning Algorithms to predict if a passenger survived from the sinking Titanic or not. So it's a binary classification problem. 

Keep Learning,

vikas singh


# CONTEXT
* [1. Importing packages and Collecting Data](#1)
* [2. Variable Description and Identification](#2)
  * [2.1 Categorical and Numerical Variables ](#2.1)
  * [2.2 Variable Data Types](#2.2)
  * [2.3 Data Description](#2.3)
  * [2.4 Missing Variables](#2.4)
* [3. Feature Engineering Or Data Preprocessing](#3)
  * [3.1 Process Cabin](#3.1)
  * [3.2 Process Name](#3.2)
  * [3.3 Process SibSp & Parch](#3.3)
  * [3.4 Process Ticket](#3.4)
  * [3.5 Outliers Detection](#3.5)
    * [3.5.1 Outliers detection for Fare](#3.5.1)
    * [3.5.1 Outliers detection for Age](#3.5.2)
  * [3.6 Imputing Missing Variables](#3.6)
     * [3.6.1 Imputing Embarked and Fare](#3.6.1)
     * [3.6.2 Impute Age](#3.6.2)
  * [3.7 Data Transformation](#3.7)
      * [3.7.1 Binning Age](#3.7.1)
      * [3.7.2 Binning Fare](#3.7.2)
  * [3.8 Correcting Data Type](#3.8)
  * [3.9 Dropping Features](#3.9)
  * [3.10 Encoding Categorical Variables](#3.10)
* [4.Model Building and Evaluation](#4)
  * [4.1 Training Model](#4.1)
  * [4.2 Cross-validation: Evaluating estimator performance](#4.2)
    * [4.2.1 K-Fold Cross Validation](#4.2.1)
    * [4.2.2 Tuning Hyperparameters](#4.2.2)
  * [4.3 Retrain and Predict Using Optimized Hyperparameters](#4.3)
*  [5.Prediction & Submission](#5)
*  [6. Introduction to Ensemble](#6)
   * [6.1 Different Ensemble Methods](#6.1)
     * [6.1.1 Simple Ensemble Methods](#6.1.1)
     * [6.1.2 Advanced Ensemble Methods](#6.1.2)

# 1. Importing packages and Collecting Data <a id="1"></a>

In [ ]:
# Data Manipulattion
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Importing Dependencies
%matplotlib inline

# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read and preview the train data from csv file.
train = pd.read_csv('../input/train.csv')
train.head(3)

In [ ]:
# Read and preview the test data from csv file.
test = pd.read_csv('../input/test.csv')
train.head(3)

# 2. Variable Description and Identification <a id="2"></a>
Describe what each of the variable indicates and identify our response and predictor variables. Then seperate the categorical variables from numerical variables and finally identify pandas data types (i.e., object, float64 or int64) for every variable.

In [ ]:
# Merge train and test data together. This eliminates the hassle of handling train and test data seperately for various analysis.
merged = pd.concat([train,test], sort = False)
merged.head(3)

In [ ]:
# Let's see the shape of the combined data
merged.shape

In [ ]:
# variable in the combined data
merged.columns

### So what can we see??
**We can see total 12 variables. And each variable has 1309 observations (excluding Survived).**
### Here comes the description of all variables:
1. **PassengerId** is a unique identifying number assigned to each passenger.
2. **Survived** is a flag that indicates if a passenger survived or died ( i.e., 0 = No, 1 = Yes).
3. **Pclass** is the passenger class (i.e., 1 = 1st class, 2 = 2nd class, 3 = 3rd class).
4. **Name** is the name of the passenger.
5. **Sex** indicates the gender of the passenger (i.e., Male or female).
6. **Age** indicates the age of the passenger.
7. **Sibsp**  is the number of siblings/spouses aboard.
8. **Parch** is the number of parents/children aboard.
9. **Ticket** indicates the ticket number issued to the passenger.
10. **Fare** indicates the amount of money spent on their ticket.
11. **Cabin** indicates the cabin category occupied by the passenger.
12. **Embarked** indicates the port where the passenger embarked from (i.e., C = Cherbourg, Q = Queenstown, S = Southampton).


### Here, Survived is the target variable and rest of the variables are predictor variables.

## 2.1 Categorical and Numerical Variables  <a id="2.1"></a>
**Categorical Variable:** Survived, Sex, Pclass (ordinal), Embarked, Cabin, Name, Ticket, SibSp, and Parch.

**Numerical Variable:** Fare, Age, and PassengerId.

## 2.2 Variable Data Types <a id="2.1"></a>


In [ ]:
# data types of different variables
merged.info()

  * Three float variables: - Age, Fare, and Survived.
  * Four int variables: - Pclass, SibSp, Parch and PassengerId
  * Five Object(number+strings) variables: -  Name, Sex, Ticket, Cabin,       and Embarked.

## 2.3 Data Description <a id="2.3"></a>

In [ ]:
# Description of the data variables
merged.describe()

## 2.4 Missing Variables <a id="2.4"></a>

In [ ]:
# Visualization of Missing variables
plt.figure(figsize=(8,4))
sns.heatmap(merged.isnull(), yticklabels=False, cbar=False, cmap='plasma')

In [ ]:
# Count of missing variables
merged.isnull().sum()

  * Survived column is showing missing values becuase in test.csv don't       have the survived column.
  * Age, Cabin, Fare and Embarked has missing values.

# 3. Feature Engineering Or Data Preprocessing <a id="3"></a>
In this section, we transform raw data into understandable format.  We would engineer features like Cabin, Name, SibSp & Parch, and Ticket that could tell us something about survival or death once they're processed.

## 3.1 Process Cabin <a id="3.1"></a>

In [ ]:
# let's preview the cabin again.
merged['Cabin'].head()

In [ ]:
# we see that Cabin contains some missing values. let's count it again.
merged['Cabin'].isnull().sum()

In [ ]:
# Let's manully understand the Cabin column.
merged['Cabin'].value_counts().head()

  * Looks like Cabin is alphanumeric type variable with no special characters (like ., /, % etc) between letters and numbers.
  * It has also 1014 missing obsevations.
  * It is reasonable to presume that those NaNs didn't have a cabin.
  * We will flag NaN as 'X' and keep only the 1st character where Cabin has alphanumeric values.

In [ ]:
# let's fill all NaNs of cabin as 'X'
merged['Cabin'].fillna(value = 'X', inplace=True)

In [ ]:
# Keeping 1st charater from the Cabin
merged['Cabin'] = merged['Cabin'].apply(lambda x: x[0])
merged['Cabin'].value_counts()

## 3.2 Process Name <a id="3.2"></a>

In [ ]:
#Let's see the Name column.
merged['Name'].head(10)

  * This column contains string that furth contains titles such as Mr, Mrs, Master etc.
  * These title give us useful information about sex and age for example Mr=Male, Mrs=Female and married, miss= Female and young.
  * Now we want to extract these titles from Name to check if there is any association between these titles and Survived.

In [ ]:
# Extracting title from Name and create a new variable Title.
merged['Title'] = merged['Name'].str.extract('([A-Za-z]+)\.')
merged['Title'].head()

In [ ]:
# let's see the different categories of Title from Name column.
merged['Title'].value_counts()

  * We can see there are several titles with the very least frequency. So, it makes sense to put them in fewer buckets.
  * Professionals like Dr, Rev, Col, Major, Capt will be put into 'Officer' bucket.
  * Titles such as Dona, Jonkheer, Countess, Sir, Lady, Don were usually entitled to the aristocrats.
  * We would also replace Mlle and Ms with Miss and Mme by Mrs as these are French titles.

In [ ]:
# Replacing  Dr, Rev, Col, Major, Capt with 'Officer'
merged['Title'].replace(to_replace = ['Dr', 'Rev', 'Col', 'Major', 'Capt'], value = 'Officer', inplace=True)

# Replacing Dona, Jonkheer, Countess, Sir, Lady with 'Aristocrate'
merged['Title'].replace(to_replace = ['Dona', 'Jonkheer', 'Countess', 'Sir', 'Lady', 'Don'], value = 'Aristocrat', inplace = True)

#  Replace Mlle and Ms with Miss. And Mme with Mrs.
merged['Title'].replace({'Mlle':'Miss', 'Ms':'Miss', 'Mme':'Mrs'}, inplace = True)

In [ ]:
# let's see how Tittle looks now
merged['Title'].value_counts()

## 3.3 Process SibSp & Parch <a id="3.3"></a>
 Since these two variables together indicate the size of a family, we would create a new variable 'Family_size' from these two variables.

In [ ]:
# Merging Sibsp and Parch and creating new variable called 'Family_size'
merged['Family_size'] = merged.SibSp + merged.Parch + 1  # Adding 1 for single person
merged['Family_size'].value_counts()

  * We see there are several family sizes with the very least frequency like 2,3 and some have large frequency 7,8,11
  * We will create 4 buckets namely single, small, medium, and large for rest of them.

In [ ]:
# Create buckets of single, small, medium, and large and then put respective values into them.
merged['Family_size'].replace(to_replace = [1], value = 'single', inplace = True)
merged['Family_size'].replace(to_replace = [2,3], value = 'small', inplace = True)
merged['Family_size'].replace(to_replace = [4,5], value = 'medium', inplace = True)
merged['Family_size'].replace(to_replace = [6, 7, 8, 11], value = 'large', inplace = True)

In [ ]:
# let's see how 'Family_size' looks now
merged['Family_size'].value_counts()

## 3.4 Process Ticket <a id="3.4"></a>

In [ ]:
# let's preview the Ticket variable.
merged['Ticket'].head(10)

  * Ticket variable contains alphanumeric, only numbers and character type variables.
  *  We will create two groups-one will contain just number and other will only contain character extracted from string.
  * And assign 'N' to the number type variable.
  * If a row contains both character and number, we will keep only character.
  

In [ ]:
# Assign N if there is only number and no character. If there is a character, extract the character only.
ticket = []
for x in list(merged['Ticket']):
    if x.isdigit():
        ticket.append('N')
    else:
         ticket.append(x.replace('.','').replace('/','').strip().split(' ')[0])
# Swap values
merged['Ticket'] = ticket

In [ ]:
# Let's count the categories in  Ticket
merged['Ticket'].value_counts()

In [ ]:
# Keeping only the 1st character to reduce the Ticket categories
merged['Ticket'] = merged['Ticket'].apply(lambda x : x[0])
merged['Ticket'].value_counts()

## 3.5 Outliers Detection <a id="3.5"></a>
In statistics, an outlier is a data point that significantly differs from the other data points in a sample. Often, outliers in a data set can alert statisticians to experimental abnormalities or errors in the measurements taken, which may cause them to omit the outliers from the data set. If they do omit outliers from their data set, significant changes in the conclusions drawn from the study may result.

**See the data Description table above for  min, 1st quartile, 2nd quartile(median), 3rd quartile, and max values of a variable.**

**We will use IQR method to detect the outliers for variable Age and Fare though we won't remove them.**


In [ ]:
# Create a function to count total outliers.
def outliers(variable):
    global filtered # Global keyword is used inside a function only when we want to do assignments or when we want to change a variable.
    
    # Calculate 1st, 3rd quartiles and iqr.
    q1, q3 = variable.quantile(0.25), variable.quantile(0.75)
    iqr = q3 - q1
    
    # Calculate lower fence and upper fence for outliers
    l_fence, u_fence = q1 - 1.5*iqr , q3 + 1.5*iqr   # Any values less than l_fence and greater than u_fence are outliers.
    
    # Observations that are outliers
    outliers = variable[(variable<l_fence) | (variable>u_fence)]
    print('Total Outliers of', variable.name,':', outliers.count())
    
    # Drop obsevations that are outliers
    filtered = variable.drop(outliers.index, axis = 0)

### 3.5.1 Outliers detection for Fare <a id="3.5.1"></a>

In [ ]:
# Total number of outliers in Fare
outliers(merged['Fare'])

In [ ]:
# Visualisation of Fare distribution with outliers
plt.figure(figsize=(13, 2))
sns.boxplot(x=merged["Fare"],palette='Blues')
plt.title('Fare distribution with outliers', fontsize=15 )

In [ ]:
# Visualisation of Fare distribution without outliers
plt.figure(figsize=(13, 2))
sns.boxplot(x=filtered,palette='Blues')
plt.title('Fare distribution without outliers', fontsize=15 )

### 3.5.2 Outliers detection for Age <a id="3.5.2"></a>

In [ ]:
# Total number of outliers in Age
outliers(merged['Age'])

In [ ]:
# Visualisation of Age distribution with outliers
plt.figure(figsize=(13, 2))
sns.boxplot(x=merged["Age"],palette='Blues')
plt.title('Age distribution with outliers', fontsize=15)

In [ ]:
# Visualisation of Age distribution without outliers
plt.figure(figsize=(13, 2))
sns.boxplot(x=filtered,palette='Blues')
plt.title('Age distribution without outliers', fontsize=15)

## 3.6 Imputing Missing Variables <a id="3.6"></a>
There is various techniques to imputing missing variable in the datasets for exmaple linear regession, K-NN, Mean/mode/median, Deep learning etc. The simpliest way to impute missing values of a variable is to impute its missing values with its mean, median or mode depending on its distribution and variable type(categorical or numerical). 

However, one clear disadvantage of using mean, median or mode to impute missing values is the addition of bias if the amount of missing values is significant (like Age). So simply replacing them with the mean or the median age might not be the best solution since the age may differ by groups and categories of passengers.

To solve this, we can group our data by some variables that have no missing values and for each subset compute the median age to impute the missing values.

In [ ]:
# let's count the missing values for each variable
merged.isnull().sum()

 * Survived column is showing missing values becuase in test.csv don't       have the survived column.
 * Age, Cabin, Fare and Embarked has missing values.

### 3.6.1 Imputing Embarked and Fare <a id="3.6.1"></a>

In [ ]:
# imputing Embarked with mode because Embarked is a categorical variable.
merged['Embarked'].value_counts()

In [ ]:
# Here S is the most frequent
merged['Embarked'].fillna(value = 'S', inplace = True)

In [ ]:
# Impute missing values of Fare. Fare is a numerical variable with outliers. Hence it will be imputed by median.'''
merged['Fare'].fillna(value = merged['Fare'].median(), inplace = True)

### 3.6.2 Impute Age <a id="3.6.2"></a>
To impute Age with grouped median, we need to know which features are heavily correlated with Age. Let's find out the variables correlated with Age.

In [ ]:
# Let's plot correlation heatmap to see which variable is highly correlated with Age. We need to convert categorical variable into numerical to plot correlation heatmap. So convert categorical variables into numerical.
df = merged.loc[:, ['Sex', 'Pclass', 'Embarked', 'Title', 'Family_size', 'Parch', 'SibSp', 'Cabin', 'Ticket']]
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df = df.apply(le.fit_transform) # data is converted.
df.head(2)

In [ ]:
 # Inserting Age in variable correlation.
df['Age'] = merged['Age']
# Move Age at index 0.
df = df.set_index('Age').reset_index()
df.head(2)

In [ ]:
# Now create the heatmap correlation of df
plt.figure(figsize=(10,6))
sns.heatmap(df.corr(), cmap ='BrBG',annot = True)
plt.title('Variables correlated with Age')
plt.show()

  * Sex, Embarked, and Ticket have the weakest correlation with Age.
  * Pclass and Title have strong correlation with age.
  **So the tactic is to impute missing values of Age with the median age of similar rows according to Title and Pclass.**

In [ ]:
# Create a boxplot to view the correlated and medium of the Pclass and Title variables with Age.
# Boxplot b/w Pclass and Age
sns.boxplot(y='Age', x='Pclass', data=merged)

In [ ]:
# Boxplot b/w Title and Age
sns.boxplot(y='Age', x='Title', data=merged)

In [ ]:
# Impute Age with median of respective columns (i.e., Title and Pclass)
merged['Age'] = merged.groupby(['Title', 'Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))

In [ ]:
# let's check the missing value again.
merged.isnull().sum()

## 3.7 Data Transformation <a id="3.7"></a>
In this section, we will transform our continuous variables. After that, redundant and useless features will be deleted. And finally categorical variables will be encoded into numerical to feed our machine learning models.

### 3.7.1 Binning Age <a id="3.7.1"></a>

In [ ]:
# Creating bin categories for Age 
label_names = ['infant', 'child', 'teenager','young_adult', 'adult', 'aged']

# Create range for each bin categrories of age
cut_points = [0,5,12,18,35,60,81]

#Create and view categorized Age with original Age.
merged['Age_binned'] = pd.cut(merged['Age'], cut_points, labels = label_names)

#Age with Categorized Age.
merged[['Age', 'Age_binned']].head(2)

### 3.7.2 Binning Fare <a id="3.7.2"></a>

In [ ]:
# Create bin categories for Fare
groups = ['low','medium','high','very_high']

# Create range for each bin categories of Fare
cut_points = [-1, 130, 260, 390, 520]

#Create and view categorized Fare with original Fare
merged['Fare_binned'] = pd.cut(merged.Fare, cut_points, labels = groups)

# Fare with Categorized Fare
merged[['Fare', 'Fare_binned']].head(2)

#### (Optional) Standarding Fare

In [ ]:
# import scaling model
#from sklearn.preprocessing import MinMaxScaler

#Create a scaler object
#scaler = MinMaxScaler()

# Fit and transform the merged['Fare']
#merged['Fare'] = scaler.fit_transform(merged['Fare'].values.reshape(-1,1))
#merged['Fare'].head()

** we have option to do binning and standarding of continuous variables but here we did binning for get more accuary in the Desicion tree and Random forest classifier**

## 3.8 Correcting Data Type <a id="3.8"></a>

In [ ]:
# checking the data type
merged.dtypes

In [ ]:
# Correcting data types, converting into categorical variables.
merged.loc[:, ['Pclass', 'Sex', 'Embarked', 'Cabin', 'Title', 'Family_size', 'Ticket']] = merged.loc[:, ['Pclass', 'Sex', 'Embarked', 'Cabin', 'Title', 'Family_size', 'Ticket']].astype('category')

# Due to merging there are NaN values in Survived for test set observations.
merged['Survived'] = merged['Survived'].dropna().astype('int') #Converting without dropping NaN throws an error

In [ ]:
# Check if data types have been corrected
merged.dtypes

## 3.9 Dropping Features <a id="3.9"></a>
Now we have both transformed and the original variables transformation have been made from. So we should safely drop the variables that we think would not be useful anymore for our survival analysis since they are very unlikely to be analyzed in their raw forms.

In [ ]:
# let's see all the variables
merged.head(2)

In [ ]:
# droping the feature that would not be useful anymore
merged.drop(columns = ['Name', 'Age','SibSp', 'Parch','Fare'], inplace = True, axis = 1)
merged.columns

## 3.10 Encoding Categorical Variables <a id="3.10"></a>
We would like to use one hot encoding instead of label encoding. a one-hot encoding can be applied to the integer representation. This is where the integer encoded variable is removed and a new binary variable is added for each unique integer value.

In [ ]:
# convert categotical data into dummies variables
merged = pd.get_dummies(merged, drop_first=True)
merged.head(2)

# 4.Model Building and Evaluation <a id="4"></a>

In [ ]:
#Let's split the train and test set to feed machine learning algorithm.
train = merged.iloc[:891, :]
test  = merged.iloc[891:, :]

In [ ]:
#Drop passengerid from train set and Survived from test set.'''
train = train.drop(columns = ['PassengerId'], axis = 1)
test = test.drop(columns = ['Survived'], axis = 1)

In [ ]:
# setting the data as input and output for machine learning models
X_train = train.drop(columns = ['Survived'], axis = 1) 
y_train = train['Survived']

# Extract test set
X_test  = test.drop("PassengerId", axis = 1).copy()

In [ ]:
# See the dimensions of input and output data set.'''
print('Input Matrix Dimension:  ', X_train.shape)
print('Output Vector Dimension: ', y_train.shape)
print('Test Data Dimension:     ', X_test.shape)

## 4.1 Training Model <a id="4.1"></a>
We would train 5 different classifiers for this binary classification problem.

In [ ]:
# Now initialize all the classifiers object.

#1.Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

#2.KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

#3.Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 40)

#4.Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 40, n_estimators = 100)

#5.Support Vector Machines
from sklearn.svm import SVC
svc = SVC(gamma = 'auto')

#6. XGBoost 
from xgboost import XGBClassifier
xgb = XGBClassifier(n_job = -1, random_state = 40)

In [ ]:
# Create a function that returns train accuracy of different models.

def train_accuracy(model):
    model.fit(X_train, y_train)
    train_accuracy = model.score(X_train, y_train)
    train_accuracy = np.round(train_accuracy*100, 2)
    return train_accuracy
    
# making the summary table of train accuracy.
train_accuracy = pd.DataFrame({'Train_accuracy(%)':[train_accuracy(lr), train_accuracy(knn), train_accuracy(dt), train_accuracy(rf), train_accuracy(svc), train_accuracy(xgb)]})
train_accuracy.index = ['LR', 'KNN','DT', 'RF', 'SVC', 'XGB']
sorted_train_accuracy = train_accuracy.sort_values(by = 'Train_accuracy(%)', ascending = False)

#Training Accuracy of the Classifiers
sorted_train_accuracy

**IN the above table, DT, RF, XGB, KNN models have highest train accuracy. But train accuracy of a model is not enough to tell if a model can be able to generalize the unseen data or not.
we can't use training accuracy for our model evaluation rather we must know how our model will perform on the data our model is yet to see.**

## 4.2 Cross-validation: Evaluating estimator performance <a id="4.2"></a>
Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called **overfitting.**

One thing we can do is to split the train set in two groups, usually in 80:20 ratio. That means we would train our model on 80% of the training data and we reserve the rest 20% for evaluating the model since we know the ground truth for this 20% data. This is the first model evaluation technique. In sklearn we have a train_test_split method for that.

But Train_test split has its drawbacks. Because this approach introduces bias as we are not using all of our observations for testing and also we're reducing the train data size. To overcome this we can use a technique called **cross validation** where all the data is used for training and testing periodically.

However, as the train set gets larger, train_test_split has its advantage over k-fold cross validation. Train_test_split is k-times faster than k-fold cross validation. If the training set is very large, both train_test_split and k-fold cross validation perform identically. So for a large training data, train_test_split is prefered over k-fold cross validation to accelerate the training process.

### 4.2.1 K-Fold Cross Validation ¶<a id="4.2.1"></a>

Let's say we will use 10-fold cross validation. So k = 10 and we have total 891 observations. Each fold would have 891/10 = 89.1 observations. So basically k-fold cross validation uses fold-1 (89.1 samples) as the testing set and k-1 (9 folds) as the training sets and calculates test accuracy.This procedure is repeated k times (if k = 10, then 10 times); each time, a different group of observations is treated as a validation or test set. This process results in k estimates of the test accuracy which are then averaged out.

In [ ]:
# Create a function that returns mean cross validation score for different models.
def val_score(model):
    from sklearn.model_selection import cross_val_score
    val_score = cross_val_score(model, X_train, y_train, cv = 10, scoring = 'accuracy').mean()
    val_score = np.round(val_score*100, 2)
    return val_score

# making the summary table of cross validation accuracy.
val_score = pd.DataFrame({'val_score(%)':[val_score(lr), val_score(knn), val_score(dt), val_score(rf), val_score(svc), val_score(xgb)]})
val_score.index = ['LR', 'KNN','DT', 'RF', 'SVC', 'XGB']
sorted_val_score = val_score.sort_values(by = 'val_score(%)', ascending = False)

#cross validation accuracy of the Classifiers
sorted_val_score

**In the above tabel, LR and KNN have the highest cross validation accuracy among the remaining models.**

### 4.2.2 Tuning Hyperparameters <a id="4.2.2"></a>
Now let's add Grid Search to all the classifiers with the hopes of optimizing their hyperparameters and thus **improving their accuracy**. Are the default model parameters the best bet? Let's find out.

**For more detail visit on sklearn documention for individual models.**


In [ ]:
# define all the model hyperparameters one by one first

# 1. For logistic regression
lr_params = {'penalty':['l1', 'l2'],
             'C': np.logspace(0, 2, 4, 8 ,10)}

# 2. For KNN
knn_params = {'n_neighbors':[4,5,6,7,8,9,10],
              'weights':['uniform', 'distance'],
              'algorithm':['auto', 'ball_tree','kd_tree','brute'],
              'p':[1,2]}

# 3. For DT
dt_params = {'max_features': ['auto', 'sqrt', 'log2'],
             'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 
             'min_samples_leaf':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
             'random_state':[46]}
# 4. For RF
rf_params = {'criterion':['gini','entropy'],
             'n_estimators':[ 10, 30, 200, 400],
             'min_samples_leaf':[1, 2, 3],
             'min_samples_split':[3, 4, 6, 7], 
             'max_features':['sqrt', 'auto', 'log2'],
             'random_state':[46]}
# 5. For SVC
svc_params = {'C': [0.1, 1, 10,100], 
              'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
              'gamma': [ 1, 0.1, 0.001, 0.0001]}

#6. For XGB
xgb_params = xgb_params_grid = {'min_child_weight': [1, 5],
                   'gamma': [0.04, 0, 0.1, 1.5],
                   'subsample': [0.6, 0.8, 1.0],
                   'colsample_bytree': [0.46, 1.0],
                   'max_depth': [3, 7]}

In [ ]:
# Create a function to tune hyperparameters of the selected models.
def tune_hyperparameters(model, param_grid):
    from sklearn.model_selection import GridSearchCV
    global best_params, best_score #if you want to know best parametes and best score
    
    # Construct grid search object with 10 fold cross validation.
    grid = GridSearchCV(model, param_grid, verbose = 3, cv = 10, scoring = 'accuracy', n_jobs = -1)
    # Fit using grid search.
    grid.fit(X_train, y_train)
    best_params, best_score = grid.best_params_, np.round(grid.best_score_*100, 2)
    return best_params, best_score

In [ ]:
# Appling tune hyperparameters in the created funtion

# Tune LR hyperparameters.
tune_hyperparameters(lr, param_grid=lr_params)
lr_best_params, lr_best_score =  best_params, best_score
print('LR Best Score:', lr_best_score)
print('And Best Parameters:', lr_best_params)

In [ ]:
# Tune KNN hyperparameters
tune_hyperparameters(knn, param_grid=knn_params)
knn_best_params, knn_best_score =  best_params, best_score

In [ ]:
# Tune DT hyperparameters
tune_hyperparameters(dt, param_grid=dt_params)
dt_best_params, dt_best_score =  best_params, best_score

In [ ]:
# Tune RF hyperparameters
tune_hyperparameters(rf, param_grid=rf_params)
rf_best_params, rf_best_score =  best_params, best_score

In [ ]:
# Tune SVC hyperparameters
tune_hyperparameters(svc, param_grid=svc_params)
svc_best_params, svc_best_score =  best_params, best_score

In [ ]:
# Tune XGB hyperparameters
xgb_opt = XGBClassifier(learning_rate = 0.04, n_estimators = 500, 
                       silent = 1, nthread = -1, random_state = 101)
tune_hyperparameters(xgb_opt, param_grid=xgb_params)
xgb_best_params, xgb_best_score =  best_params, best_score

In [ ]:
# lets compares cross validation scores with tunned scores for different models.
# Create a dataframe of tunned scores and sort them in descending order.'''
tunned_scores = pd.DataFrame({'Tunned_accuracy(%)': [lr_best_score, knn_best_score, dt_best_score, rf_best_score, svc_best_score, xgb_best_score]})
tunned_scores.index = ['LR', 'KNN', 'DT', 'RF', 'SVC', 'XGB']
sorted_tunned_scores = tunned_scores.sort_values(by = 'Tunned_accuracy(%)', ascending = False)
# Models Accuracy after Optimization
sorted_tunned_scores

when we copares the cross validation scores with tunned scores we can see that all the classifier are improved.  Among the classifiers, RF and SVC have the highest accuracy after tunning hyperparameters.

## 4.3 Retrain and Predict Using Optimized Hyperparameters <a id="4.3"></a>

So we have our best classifiers with their best hyperparameters that produces best accuracy out of a model. That means if we retrain the classifiers using their best hyperparameters, we will be able to get the very same score that we got after tunning hyperparameters (see part 14.4). Let's retrain our classifiers and then use cross validation to calculate the accuracy of the trained model. 

In [ ]:
# Instantiate the models with optimized hyperparameters.
lr  = LogisticRegression(**lr_best_params)
knn = KNeighborsClassifier(**knn_best_params)
dt  = DecisionTreeClassifier(**dt_best_params)
rf  = RandomForestClassifier(**rf_best_params)
svc = SVC(**svc_best_params)
xgb = XGBClassifier(**xgb_best_params)

In [ ]:
# Train all the models with optimised hyperparameters
models = { 'LR': lr, 'KNN':knn,'DT':dt,'RF':rf, 'SVC':svc, 'XGB':xgb}

# 10-fold Cross Validation after Optimization
score = []
for x, (keys, items) in enumerate(models.items()):
    # Train the models with optimized parameters using cross validation.
    # No need to fit the data. cross_val_score does that for us.
    # But we need to fit train data for prediction in the follow session.
    from sklearn.model_selection import cross_val_score
    items.fit(X_train, y_train)
    scores = cross_val_score(items, X_train, y_train, cv = 10, scoring = 'accuracy')*100
    score.append(scores.mean())
    print('Mean Accuracy: %0.4f (+/- %0.4f) [%s]'  % (scores.mean(), scores.std(), keys))

In [ ]:
# Make prediction using all the trained models
model_prediction = pd.DataFrame({'LR':lr.predict(X_test), 'KNN':knn.predict(X_test), 'DT':dt.predict(X_test),'RF':rf.predict(X_test), 'SVC':svc.predict(X_test), 'XGB': xgb.predict(X_test)})

#All the Models Prediction 
model_prediction.head()

# 5. Prediction & Submission <a id="5"></a>
we will predict using both rf and svc. Then we will create two prediction files in csv format for kaggle submission.

In [ ]:
# Submission with the most accurate random forest classifier
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": rf.predict(X_test)})
submission.to_csv('submission_rf.csv', index = False)


# Submission with the most accurate SVC classifier.
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": svc.predict(X_test)})
submission.to_csv('submission_svc.csv', index = False)

# Submission with the most accurate XGB classifier.
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": xgb.predict(X_test)})
submission.to_csv('submission_xgb.csv', index = False)

# 6. Introduction to Ensemble <a id="6"></a>
Can we further boost the accuracy of our best models? That's what we will try to do using ensemble method. Ensembles combine predictions from different models to generate a final prediction, and the more models we include the better it performs. Better still, because ensembles combine baseline predictions, they perform at least as well as the best baseline model. Most of the errors from a model’s learning are from three main factors: variance, noise, and bias. By using ensemble methods, we’re able to increase the stability of the final model and reduce the errors caused by bias, variance, and noise. By combining many models, we’re able to (mostly) reduce the variance, even when they are individually not great, as we won’t suffer from random errors from a single source. **The main principle behind ensemble modelling is to group weak learners together to form one strong learner. The most basic ensemble is majority voting rule (where the prediction or vote given by the majority of the models used as final prediction).But there are many other ways to combine predictions, and more generally we can use a model to learn how to best combine predictions.**

**To implement an ensemble we need three basic things:**
1. A group of base learners that generate predictions.
2. A meta learner that learns how to best combine these predictions outputed by base learners.
3. And finally a method for splitting the training data between the base learners and the meta learner.

**An ensemble works best if:**
1. There is a less correlation in the base models' predictions.
2. We increase the number of base learners though it might slow the process down.

![](https://cdn-images-1.medium.com/max/1000/0*sOtXk_8ZftGGU00_.png)

## 6.1 Different Ensemble Methods <a id="6.1"></a>
We would first categorize ensemble methods into two subcategories like 1.Simple Ensemble Methods and 2.Advanced Ensemble Methods

### 6.1.1 Simple Ensemble Methods <a id="6.1.1"></a>
They're the simpliest yet so useful form of enselbles. They can be further categorised into 
1. Voting, 
2. Averaging and 
3. Weighted Average. 

First one is usually used for classification while the later two are used for regression problems.

**Voting Ensemble**  
Voting ensemble is further classified into 
1. Hard voting and 
2. Soft voting.

#### 6.1.1.1 Hard Voting (or Majority Voting or Max Voting) <a id="6.1.1.1"></a>
This hard voting method is usually used for classification problems. The idea is to train multiple models to make predictions for each data point. The predictions by each model are considered as a ‘vote’. The predictions which we get from the majority of the models are used as the final prediction. Say rf and lr predict a class as 1 while knn predicts the same class as 0. Since the majority of the vots is casted in favour of class 1, the voting classifier would predict the very same class as 1. See the table below to understand how hard voting ensemble works.****

In [ ]:
# Example : How hard voting works
data =[[1, 1, 1, 0, 1],
       [0, 0, 0, 1, 0]]
display(pd.DataFrame(data, columns = ['Class', 'RF', 'LR', 'KNN', 'Hard_voting']).set_index('Class'))

**Correlation among Base Models Predictions:** How base models' predictions are correlated? If base models' predictions are weakly correlated with each other, the ensemble will likely to perform better. On the other hand, for a strong correlation of predictions among the base models, the ensemble will unlikely to perform better. To sumarize, diversity of predictions among the base models is inversely proportional to the ensemble accuracy. Let's make prediction for the test set.

In [ ]:
#Create a data frame to store base modles prediction 
base_prediction = model_prediction # we have already make the data frame above of all the models prediction

base_prediction.head()

In [ ]:
#Let's visualize the correlations among the predictions of base models.
plt.figure(figsize = (15,8))
sns.heatmap(base_prediction.corr(), annot=True)
plt.title('Prediction correlation ammong the Base Models', fontsize = 18)

**Findings:** The prediction looks quite similar for the 6 classifiers except when DT is compared to the others classifiers. Now we will create an ensemble with the base models RF, XGB, DT, KNN, and LR. This ensemble can be called heterogeneous ensemble since we have three tree based, one kernel based and one linear models. We would use **EnsembleVotingClassifier method from mlxtend.classifier module** for both hard and soft voting ensembles. The advantage is it requires lesser codes to plot decision regions and I find it a bit faster than sklearn's voting classifier.

In [ ]:
# We will use mlxtend library to train, predict and plot decision regions of hard voting ensemble classifier
# Define base models for hard voting ensemble.
base_models = [lr, knn, dt, rf, xgb]

# Import ensemble classifier from mlxtend
from mlxtend.classifier import EnsembleVoteClassifier

# Initialize hard voting ensemble
hard_evc = EnsembleVoteClassifier(clfs= base_models, voting = 'hard')
print('Training Hard Voting Emsemble Classification')
display(hard_evc.fit(X_train, y_train))
print('-----Done-----')

# Predict with hard voting ensemble.
y_pred_hard_ecv = pd.DataFrame(hard_evc.predict(X_test), columns = ['HARD_ECV'])

# Hard voting cross validation score.
print('\nComputing Hard Voting Cross Val Score')
hard_x_val_score = cross_val_score(hard_evc, X_train, y_train, cv=10,  scoring = 'accuracy')
hard_x_val_score = np.round(hard_x_val_score.mean()*100,2)
print('----Done----')

# Compare hard voting score with best base models scores.
hard_vs_base_score = pd.DataFrame({'Hard_vs_base_score(%)': [hard_x_val_score, lr_best_score, knn_best_score, dt_best_score, rf_best_score, xgb_best_score]})
hard_vs_base_score.index = ['HARD_VAL_SCORE', 'LR', 'KNN', 'DT', 'RF', 'XGB']
display(hard_vs_base_score)

In [ ]:
# See base models prediction with hard voting prediction.
df_hard_base = pd.concat([base_prediction.drop('SVC', axis=1),y_pred_hard_ecv], sort = False, axis = 1)
display(df_hard_base.head(7))

**Great! We can see hard voting ensemble is considering majority of the models vote(prediction) to label a particular class. Thus it can reduce prediction errors when predicted by a single base learners.**

#### 6.1.1.2 Soft Voting
On the other hand, When an ensembles averages based on probabilities we refer to it as soft voting. In an ensemble model, all classifiers (algorithms) are able to estimate class probabilities (i.e., they all have predict_proba() method), then we can specify Scikit-Learn to predict the class with the highest probability, averaged over all the individual classifiers. In a voting classifier setting the voting parameter to 'soft' enables the models to calculate their probability(also known as confidence score) individually and present it to the voting classifier, then the voting classifier averages them and outputs the class with the highest probability. If average probablity of class-1 is greater than class-0, it outputs predicted class is 1 otherwise 0.

Note: This soft-voting classifier often work better than hard-voting as it gives more weight to highly confident votes. We Need to specify voting=”soft” and ensure that all classifiers can estimate class probabilities. One algorithm where we need to be careful is SVC, by default SVC will not give probabilities, we have to specify 'probability' hyperparameter to True. See the table below to understand how soft voting ensemble works.

In [ ]:
# Example: how soft voting works
data = [[0.49, 0.99, 0.49, 0.66, 1],
        [0.51, 0.01, 0.51, 0.34, 0]]
display(pd.DataFrame(data, columns=['RF', 'LR', 'KNN', 'Average', 'Soft Voting']))

**Let's implement soft voting ensemble in mlxtend.**

In [ ]:
# Base models for soft voting is the base models of hard voting
# Initialize soft voting ensemble
base_model = [lr, knn, dt, rf, xgb]
soft_evc = EnsembleVoteClassifier(clfs = base_model, voting = 'soft')
print('fitting soft voting ensemble')
display(soft_evc.fit(X_train, y_train))

#Predict with soft voting ensemble
y_pred_soft_evc = pd.DataFrame(soft_evc.predict(X_test), columns = ['SOFT_EVC'])

# Hard voting cross validation score
print('\nComputing Soft Voting X Val Score...')
soft_x_val_score = cross_val_score(soft_evc, X_train, y_train, cv = 10, scoring = 'accuracy')
soft_x_val_score = np.round(soft_x_val_score.mean()*100, 2)
print('----Done----')

# Compare Soft voting score with best base models scores.
soft_vs_base_score = pd.DataFrame({'Soft_Vs_Base_Score': [soft_x_val_score, lr_best_score, knn_best_score, dt_best_score, rf_best_score, xgb_best_score]})
soft_vs_base_score.index = ['SOFT_VAL_SCORE', 'LR', 'KNN', 'DT', 'RF', 'XGB']
display(hard_vs_base_score)

**NICE! Soft voting perform better than all base model.**

### 6.1.2 Advanced Ensemble Methods <a id="6.1.2"></a>
Advanced ensemble methods can further be classified into
1. Bagging
2. Boostoing
3. Stacking
4. Blending

#### 6.1.2.1 Bagging 
Bagging, is shorthand for the combination of bootstrapping and aggregating. Bootstrapping is a method to help decrease the variance of the classifier and thus reduce overfitting. So the model created should be less overfitted than a single individual model. Bagging is more suitable for high variance low bias models (complex models). Random forest itself is an ensemble machine learning algorithm that follows the bagging technique. We would use rf as the base estimator for bagging instead of default dt. Let's try to implement bagging in sklearn:

In [ ]:
# Initialize bagging classifier
from sklearn.ensemble import BaggingClassifier
bagg = BaggingClassifier(base_estimator = rf, verbose = 0, n_jobs = -1, random_state = 45)
print('Fitting Bagging Ensemble')
display(bagg.fit(X_train, y_train))
print('---Done----')

# Bagging cross validation score.
print('\nComputing Bagging X Val Score..')
bagg_x_val_score = cross_val_score(bagg, X_train, y_train, cv = 10, scoring = 'accuracy')
bagg_x_val_score = np.round(bagg_x_val_score.mean()*100, 2)
print('----Done----')

# Compare bagging ensemble score with best base models scores
bagg_vs_base_score = pd.DataFrame({'Bagging_vs_Base_Score': [bagg_x_val_score,lr_best_score, knn_best_score, dt_best_score, rf_best_score, xgb_best_score]})
bagg_vs_base_score.index = ['BAGG', 'LR', 'KNN', 'DT', 'RF', 'XGB']
display(bagg_vs_base_score)

**Findings:** Bagging can't beat our best base learners.

#### 6.1.2.2 Boosting 
Boosting refers to any Ensemble method that can combine several weak learners into a strong learner. It does this through a weighted majority vote (classification) or a weighted sum (regression). Ada boost and Gradient boost, and Extreme gradient boost are popular models that uses boosting technique. Boosting is particularly suitable for low variance high bias models (less complex models). Unlike bagging, its a sequential ensemble technique. We will perform a simple voting ensemble of boosting classifiers rather performing boosting ensemble using only a single classifer with a base estimator. **But I used only one boosting model i.e., XGB, so this boosting ensemble method will be ineffective. For appling boosting ensemble method you need three to four boosting models.**

#### 6.1.2.3 Blending 
In blending, full training data is split into training and prediction sets. The base models (also called level 0 models) are trained on this train set and then predictions are made on this prediction set. These predictions made by base learers are then fed as an input to the meta learner (also called level 1 model). That is meta learner are trained with the output (predictions) of base learners. Blending ensemble uses only a subset of data to train base learners and another subset of data to make predictions. By only fitting every base learner once on a subset of the full training data, Blend ensemble is a fast ensemble that can handle very large datasets simply by only using portion of it at each stage. The cost of this approach is that information is thrown out at each stage, as one layer will not see the training data used by the previous layer. **We will use BlendEnsemble method from mlens.ensemble module to perform blending.**

In [ ]:
# Perform blending in mlens
from mlens.ensemble import BlendEnsemble

# Initialize blend ensembler
blend = BlendEnsemble(n_jobs = -1, test_size = 0.5, random_state = 45)

# Base models for blending.
base_models = [rf, dt, knn, xgb]
blend.add(base_models)

# Meta learner for blending. We will use lr.'''
blend.add_meta(lr)

# Train the blend ensemble.
print('Fitting Blending...')
display(blend.fit(X_train, y_train))
print('----Done----')

#### 6.1.2.4 Stacking (Or Stacked Generalization) 
In blending, we trained the base learners and the meta learner on only half the data, so a lot of information is lost. To prevent this, we need to use a cross-validation strategy. Fitting an ensemble with cross-validation is often referred to as stacking, while the ensemble itself is known as the Super Learner. So basically in stacking, the individual classification models (or base models) are trained on the complete training set; then, the meta-classifier is fitted on the outputs (predictions) of those base learners. The meta-classifier can either be trained on the predicted class labels or probabilities from the ensemble.

**The basic difference between blending and stacking is therefore that stacking allows both base learners and the meta learner to train on the full data set.The outcome of stacking is improved accuracy which is typical for small and medium-sized data sets, where the effect of blending can be severe. As the data set size increases, blending and stacking performs similarly and hence for large data sets blending is preferred over stacking since stacking takes significant amount of time to train the ensemble. We will use package vecstack to perform stacking that can save you from writing a lot of codes if you implement stacking from scratch.**

In [ ]:
# Import stacking method from vecstack
from vecstack import stacking
from sklearn.metrics import accuracy_score

# Initialize base models. We will use the same base models as blending
base_models = [rf, dt, xgb, knn]

# Perform stacking
S_train, S_test = stacking(base_models,                # list of base models
                           X_train, y_train, X_test,   # data
                           regression = False,         # classification task (if you need 
                                                       # regression - set to True)
                           mode = 'oof_pred_bag',      # mode: oof for train set, predict test 
                                                       # set in each fold and vote
                           needs_proba = False,        # predict class labels (if you need 
                                                       # probabilities - set to True) 
                           save_dir = None,            # do not save result and log (to save 
                                                       # in current dir - set to '.')
                           metric = accuracy_score,    # metric: callable
                           n_folds = 10,               # number of folds
                           stratified = True,          # stratified split for folds
                           shuffle = True,             # shuffle the data
                           random_state = 45,          # ensure reproducibility
                           verbose = 1)                # print progress

**So now we have OOF from base (or 0 level models) models and we can build level 1 model. We have 4 base models (level 0 models), so we expect to get 5 columns in S_train and S_test. S_train will be our input feature to train our meta learner and then prediction will be made on S_test after we train our meta learner. And this prediction on S_test is actually the prediction for our test set (X_test). Before we train our meta learner we can investigate S_train and S_test.**

In [ ]:
print('Input features for meta learner')
display(S_train[:7])

print('Test/output (prediction set for meta learner')
display(S_test[:7])

In [ ]:
# Dimension of S_train and S_test
print('Dimension of S_train:', S_train.shape)
print('Dimension of S_test:', S_test.shape)


In [ ]:
# Initialize 1st level model that is our meta learner. We will use lr
super_learner = lr 
    
# Fit meta learner on the output of base learners
print('Fitting Stacking...')
super_learner.fit(S_train, y_train)

print('Done.')
# Finally predict using super learner.
y_pred_super = super_learner.predict(S_test)

In [ ]:
# Predicting with different ensembles

# Hard voting
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": hard_evc.predict(X_test)})
submission.to_csv('submission_hard_evc.csv', index = False)

# Soft voting
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": soft_evc.predict(X_test)})
submission.to_csv('submission_soft_evc.csv', index = False)

# Bagging
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": bagg.predict(X_test)})
submission.to_csv('submission_bagg.csv', index = False)

# Blending
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": blend.predict(X_test).astype(int)})
submission.to_csv('submission_blend.csv', index = False)

# Stacking
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": y_pred_super.astype(int)})
submission.to_csv('submission_super.csv', index = False)

**My other kernel:- **

***Simple Titanic-Exploratory Data Analysis:*** [Here](https://www.kaggle.com/vikassingh1996/titanic-exploratory-data-analysis-the-beginning)

***Extensive Data Preprocessing And Modeling*** [Here](https://www.kaggle.com/vikassingh1996/extensive-data-preprocessing-and-modeling)

**If you find my kernel useful, Some upvotes will be appreciated.**

--Thank you for reading!